In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import time

In [13]:
torch.cuda.is_available()

False

#### Please note on CPUs only  torch_dtype=torch.bfloat16 works

In [14]:
%%time
model_name = "defog/sqlcoder2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    #load_in_8bit=True,
    #load_in_4bit=True,
    device_map="auto",
    use_cache=True,
    offload_folder="."
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 6min 36s
Wall time: 3min 28s


In [15]:
question = "What is our total revenue by product in the last week?"

### The following prompt should be inserted into the question from a VectorDB

In [16]:
prompt = """### Task
Generate a SQL query to answer the following question:
`{question}`

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### SQL
Given the database schema, here is the SQL query that answers `{question}`:
```sql
""".format(question=question)

In [17]:
eos_token_id = tokenizer.eos_token_id

In [18]:
inputs = tokenizer(prompt, return_tensors="pt").to("cpu")

In [19]:
%%time
generated_ids = model.generate(
    **inputs,
    num_return_sequences=1,
    eos_token_id=eos_token_id,
    pad_token_id=eos_token_id,
    max_new_tokens=400,
    do_sample=False,
    num_beams=5,

)

CPU times: total: 4h 32min 20s
Wall time: 1h 52min 43s


In [9]:
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

In [10]:
import sqlparse

In [11]:
print(sqlparse.format(outputs[0].split("```sql")[-1], reindent=True))


SELECT p.name AS product_name,
       SUM(s.quantity * ps.supply_price) AS total_revenue
FROM sales s
JOIN products p ON s.product_id = p.product_id
JOIN product_suppliers ps ON p.product_id = ps.product_id
WHERE s.sale_date >= (CURRENT_DATE - interval '1 week')
GROUP BY p.name;
